# SQL CARTO QUERY FUNCTION
# written for PUI2016_Python2

In [3]:
from __future__ import print_function
__author__ = "Sung Hoon Yang, CUSP NYU 2018"
import numpy as np
import pandas as pd
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 88}

matplotlib.rc('font', **font)
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline
np.random.seed(999)

"""
Set up PUIDATA env var
"""
import os
os.environ["PUIDATA"] = "%s/fall18/PUI/PUIDATA"% os.getenv("HOME")
__PUIDATA_PATH__ = os.environ["PUIDATA"]

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


##### This notebook is set up to link as a default to the fb55 account.  To turn in the homework use the citibike dataset - check that you are able to read in about 40k rows

In [4]:
import sys
PY2 = True if sys.version.startswith('2') else False
SQL_SOURCE = 'https://fb55.carto.com/api/v2/sql?q='
if PY2:
    import urllib2
    import urllib
    import StringIO
    import ast
    import pandas as pd
else:
    import urllib.request as urllib2
    import urllib.parse as urllib
    import io as StringIO
    import ast
    import pandas as pd


def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        if PY2:
            response = urllib2.urlopen(source, data)
        else:
            response = urllib2.urlopen(source, bytes(data, 'utf8'))
    except urllib2.HTTPError as e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    if PY2:
        return response.read() 
    else:
        return response.read().decode('utf8')

In [5]:
test = '''
SELECT *
FROM citibike
'''

In [6]:
__RAW__ = pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')
__RAW__.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0


## Task 1 — Familiarize with SQL Clauses
• Sort data by start_station_id, tripduration
    * Only checking trips with duration <= 3 hours
• Only show the top/last 10 records (aka head and tail in SQL)
• List all unique start_station_id values
• Aggregation functions:
    * Count the number of trips (aka wc -l in SQL) 

In [27]:
sql = '''
SELECT *
    FROM citibike
    WHERE tripduration <= 60 * 60 * 3 -- tripduration is in seconds
    ORDER BY start_station_id ASC, tripduration ASC
'''

In [28]:
__RAW_DF__ = pd.read_csv(StringIO.StringIO(queryCartoDB(sql)), sep=',')
__RAW_DF__.head(10) 

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,W 52 St & 11 Ave,NaN,9059,-73.993929,107,2015-02-03 10:10:00+00,2015-02-03 10:12:00+00,72,8998,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19039,Subscriber,1980.0,2
1,NaN,W 52 St & 11 Ave,NaN,40772,-73.993929,189,2015-02-07 10:40:00+00,2015-02-07 10:43:00+00,72,40759,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,18131,Subscriber,1959.0,2
2,NaN,W 52 St & 11 Ave,NaN,40770,-73.993929,229,2015-02-07 10:40:00+00,2015-02-07 10:44:00+00,72,40757,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19729,Subscriber,1995.0,1
3,NaN,W 52 St & 11 Ave,NaN,22002,-73.993929,248,2015-02-05 06:19:00+00,2015-02-05 06:24:00+00,72,21948,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,16879,Subscriber,1970.0,1
4,NaN,W 52 St & 11 Ave,NaN,43107,-73.993929,252,2015-02-07 15:43:00+00,2015-02-07 15:47:00+00,72,43099,40.767272,515,W 43 St & 10 Ave,40.760094,-73.994618,15277,Subscriber,1983.0,1
5,NaN,W 52 St & 11 Ave,NaN,32603,-73.993929,261,2015-02-06 08:58:00+00,2015-02-06 09:02:00+00,72,32566,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,18492,Subscriber,1971.0,1
6,NaN,W 52 St & 11 Ave,NaN,16015,-73.993929,276,2015-02-04 12:46:00+00,2015-02-04 12:51:00+00,72,15971,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,19216,Subscriber,1961.0,1
7,NaN,W 52 St & 11 Ave,NaN,41366,-73.993929,283,2015-02-07 12:10:00+00,2015-02-07 12:15:00+00,72,41355,40.767272,447,8 Ave & W 52 St,40.763707,-73.985162,21588,Subscriber,1974.0,1
8,NaN,W 52 St & 11 Ave,NaN,42084,-73.993929,288,2015-02-07 13:50:00+00,2015-02-07 13:54:00+00,72,42074,40.767272,514,12 Ave & W 40 St,40.760875,-74.002777,19124,Subscriber,1978.0,1
9,NaN,W 52 St & 11 Ave,NaN,2348,-73.993929,301,2015-02-01 13:08:00+00,2015-02-01 13:13:00+00,72,2282,40.767272,447,8 Ave & W 52 St,40.763707,-73.985162,18174,Subscriber,1974.0,1


In [29]:
sql = '''
SELECT *
    FROM citibike
    ORDER BY start_station_id ASC, tripduration ASC
    LIMIT 10;
'''

In [31]:
__RAW_DF__ = pd.read_csv(StringIO.StringIO(queryCartoDB(sql)), sep=',')
__RAW_DF__.head(10)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,W 52 St & 11 Ave,NaN,9059,-73.993929,107,2015-02-03 10:10:00+00,2015-02-03 10:12:00+00,72,8998,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19039,Subscriber,1980,2
1,NaN,W 52 St & 11 Ave,NaN,40772,-73.993929,189,2015-02-07 10:40:00+00,2015-02-07 10:43:00+00,72,40759,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,18131,Subscriber,1959,2
2,NaN,W 52 St & 11 Ave,NaN,40770,-73.993929,229,2015-02-07 10:40:00+00,2015-02-07 10:44:00+00,72,40757,40.767272,72,W 52 St & 11 Ave,40.767272,-73.993929,19729,Subscriber,1995,1
3,NaN,W 52 St & 11 Ave,NaN,22002,-73.993929,248,2015-02-05 06:19:00+00,2015-02-05 06:24:00+00,72,21948,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,16879,Subscriber,1970,1
4,NaN,W 52 St & 11 Ave,NaN,43107,-73.993929,252,2015-02-07 15:43:00+00,2015-02-07 15:47:00+00,72,43099,40.767272,515,W 43 St & 10 Ave,40.760094,-73.994618,15277,Subscriber,1983,1
5,NaN,W 52 St & 11 Ave,NaN,32603,-73.993929,261,2015-02-06 08:58:00+00,2015-02-06 09:02:00+00,72,32566,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,18492,Subscriber,1971,1
6,NaN,W 52 St & 11 Ave,NaN,16015,-73.993929,276,2015-02-04 12:46:00+00,2015-02-04 12:51:00+00,72,15971,40.767272,478,11 Ave & W 41 St,40.760301,-73.998842,19216,Subscriber,1961,1
7,NaN,W 52 St & 11 Ave,NaN,41366,-73.993929,283,2015-02-07 12:10:00+00,2015-02-07 12:15:00+00,72,41355,40.767272,447,8 Ave & W 52 St,40.763707,-73.985162,21588,Subscriber,1974,1
8,NaN,W 52 St & 11 Ave,NaN,42084,-73.993929,288,2015-02-07 13:50:00+00,2015-02-07 13:54:00+00,72,42074,40.767272,514,12 Ave & W 40 St,40.760875,-74.002777,19124,Subscriber,1978,1
9,NaN,W 52 St & 11 Ave,NaN,2348,-73.993929,301,2015-02-01 13:08:00+00,2015-02-01 13:13:00+00,72,2282,40.767272,447,8 Ave & W 52 St,40.763707,-73.985162,18174,Subscriber,1974,1


In [32]:
sql = '''
SELECT *
    FROM citibike
    ORDER BY start_station_id DESC, tripduration DESC
    LIMIT 10;
'''

In [33]:
__RAW_DF__ = pd.read_csv(StringIO.StringIO(queryCartoDB(sql)), sep=',')
__RAW_DF__.head(10)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,South End Ave & Liberty St,NaN,1576,-74.015756,7225,2015-02-01 11:46:00+00,2015-02-01 13:47:00+00,3002,1506,40.711512,499,Broadway & W 60 St,40.769155,-73.981918,21202,Customer,NaN,0
1,NaN,South End Ave & Liberty St,NaN,8891,-74.015756,2139,2015-02-03 09:33:00+00,2015-02-03 10:09:00+00,3002,8831,40.711512,510,W 51 St & 6 Ave,40.760660,-73.980420,15552,Subscriber,1958.0,1
2,NaN,South End Ave & Liberty St,NaN,23568,-74.015756,2131,2015-02-05 08:42:00+00,2015-02-05 09:17:00+00,3002,23529,40.711512,359,E 47 St & Park Ave,40.755103,-73.974987,21473,Subscriber,1985.0,1
3,NaN,South End Ave & Liberty St,NaN,10232,-74.015756,1977,2015-02-03 16:04:00+00,2015-02-03 16:37:00+00,3002,10173,40.711512,318,E 43 St & Vanderbilt Ave,40.753202,-73.977987,15309,Subscriber,1969.0,1
4,NaN,South End Ave & Liberty St,NaN,27243,-74.015756,1977,2015-02-05 16:11:00+00,2015-02-05 16:44:00+00,3002,27203,40.711512,318,E 43 St & Vanderbilt Ave,40.753202,-73.977987,20069,Subscriber,1969.0,1
5,NaN,South End Ave & Liberty St,NaN,6846,-74.015756,1976,2015-02-02 09:21:00+00,2015-02-02 09:53:00+00,3002,6781,40.711512,494,W 26 St & 8 Ave,40.747348,-73.997236,19665,Subscriber,1959.0,1
6,NaN,South End Ave & Liberty St,NaN,44708,-74.015756,1920,2015-02-07 18:36:00+00,2015-02-07 19:08:00+00,3002,44703,40.711512,410,Suffolk St & Stanton St,40.720664,-73.985180,15443,Subscriber,1964.0,2
7,NaN,South End Ave & Liberty St,NaN,11648,-74.015756,1908,2015-02-03 18:54:00+00,2015-02-03 19:26:00+00,3002,11587,40.711512,279,Peck Slip & Front St,40.707873,-74.001670,19133,Subscriber,1996.0,1
8,NaN,South End Ave & Liberty St,NaN,1297,-74.015756,1900,2015-02-01 11:14:00+00,2015-02-01 11:45:00+00,3002,1228,40.711512,525,W 34 St & 11 Ave,40.755942,-74.002116,17849,Customer,NaN,0
9,NaN,South End Ave & Liberty St,NaN,17637,-74.015756,1830,2015-02-04 16:33:00+00,2015-02-04 17:04:00+00,3002,17590,40.711512,318,E 43 St & Vanderbilt Ave,40.753202,-73.977987,20246,Subscriber,1969.0,1


In [34]:
sql = '''
SELECT DISTINCT start_station_id
    FROM citibike
'''

In [35]:
__RAW_DF__ = pd.read_csv(StringIO.StringIO(queryCartoDB(sql)), sep=',')
__RAW_DF__.head(20) # top ten, followed by bottom ten

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN
5,144,NaN
6,266,NaN
7,263,NaN
8,216,NaN
9,328,NaN


In [36]:
sql = '''
SELECT count(1)
    FROM citibike
'''

In [37]:
__RAW_DF__ = pd.read_csv(StringIO.StringIO(queryCartoDB(sql)), sep=',')
__RAW_DF__.head(20) # top ten, followed by bottom ten

,count,Unnamed: 1
0,46200,NaN


## Task 2 — Working with date/time
* Selecting trips started on Feb-02-2015 only
* Selecting trips started on the weekends
     * What are average trip duration during weekends?
* Can we do the same for weekday?

In [38]:
## Selecting trips started on Feb-02-2015 only
sql = '''
SELECT *
FROM citibike
WHERE date(starttime) = '2015-02-02'
'''
__RAW_DF__ = pd.read_csv(StringIO.StringIO(queryCartoDB(sql)), sep=',')
__RAW_DF__.head(7) 

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,5 Ave & E 29 St,NaN,7433,-73.986831,376,2015-02-02 17:44:00+00,2015-02-02 17:50:00+00,474,7367,40.745168,537,Lexington Ave & E 24 St,40.740259,-73.984092,18048,Subscriber,1977.0,1
1,NaN,Allen St & Hester St,NaN,7779,-73.991908,1217,2015-02-02 20:12:00+00,2015-02-02 20:32:00+00,361,7712,40.716059,461,E 20 St & 2 Ave,40.735877,-73.982050,19732,Customer,NaN,0
2,NaN,E 5 St & Avenue C,NaN,11,-73.979955,1312,2015-02-02 11:07:00+00,2015-02-02 11:29:00+00,393,6920,40.722992,476,E 31 St & 3 Ave,40.743943,-73.979661,17720,Subscriber,1955.0,1
3,NaN,9 Ave & W 16 St,NaN,627,-74.004432,444,2015-02-02 00:40:00+00,2015-02-02 00:48:00+00,463,6477,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20233,Subscriber,1971.0,1
4,NaN,9 Ave & W 16 St,NaN,991,-74.004432,438,2015-02-02 00:40:00+00,2015-02-02 00:47:00+00,463,6478,40.742065,489,10 Ave & W 28 St,40.750664,-74.001768,20264,Subscriber,1981.0,2
5,NaN,W 33 St & 7 Ave,NaN,1037,-73.990931,498,2015-02-02 06:20:00+00,2015-02-02 06:28:00+00,492,6526,40.750200,493,W 45 St & 6 Ave,40.756800,-73.982912,16013,Subscriber,1962.0,2
6,NaN,W 27 St & 7 Ave,NaN,6510,-73.993915,199,2015-02-02 00:02:00+00,2015-02-02 00:05:00+00,442,6442,40.746647,489,10 Ave & W 28 St,40.750664,-74.001768,20684,Subscriber,1992.0,1


In [39]:
## Selecting trips started on the weekends
## Calculating the average trip duration during weekends
sql_wkdays = '''
SELECT 
    EXTRACT(ISODOW FROM date(starttime)), AVG(tripduration)
FROM citibike
WHERE EXTRACT(ISODOW FROM date(starttime)) IN (6, 7) -- 6: Saturday, 7: Sunday
GROUP BY 1
ORDER BY EXTRACT(ISODOW FROM date(starttime)) ASC
'''

In [40]:
__RAW_DF__ = pd.read_csv(StringIO.StringIO(queryCartoDB(sql_wkdays)), sep=',')
__RAW_DF__.head(2)

,date_part,avg
0,6,686.460825
1,7,638.149200


In [41]:
## Selecting trips started on the weekdays
## Calculating the average trip duration during weekdays
sql_wkdays = '''
SELECT 
    EXTRACT(ISODOW FROM date(starttime)), AVG(tripduration)
FROM citibike
WHERE EXTRACT(ISODOW FROM date(starttime)) NOT IN (6, 7) -- 6: Saturday, 7: Sunday
GROUP BY 1
ORDER BY EXTRACT(ISODOW FROM date(starttime)) ASC
'''

In [42]:
__RAW_DF__ = pd.read_csv(StringIO.StringIO(queryCartoDB(sql_wkdays)), sep=',')
__RAW_DF__.head(5)

,date_part,avg
0,1,920.862234
1,2,767.224443
2,3,697.556559
3,4,623.396720
4,5,637.116968


## Task 3 — Working with Space
• Showing the list of start station locations
    * Using GROUP BY
• Showing the number of trips started per station
• … but only for stations within 500m of Time Square!
    * The coordinates of Time Square is (40.7577,-73.9857)

In [43]:
## Selecting trips started on the weekdays
## Calculating the average trip duration during weekdays
sql = '''
    SELECT  
        start_station_name
        , COUNT(1) AS trip_count
    FROM citibike
    WHERE SQRT(
                (111.12 * 1000 * (start_station_latitude - 40.7577))^2 + 
                (111.12 * 1000 * (end_station_longitude + 73.9857) * cos(40.7577 / 92.215))^2
            ) < 500 -- within 500m of Time Square
GROUP BY 1
ORDER BY 2 DESC
'''

In [44]:
__RAW_DF__ = pd.read_csv(StringIO.StringIO(queryCartoDB(sql)), sep=',')
__RAW_DF__.head(5)

,start_station_name,trip_count
0,W 41 St & 8 Ave,141
1,W 38 St & 8 Ave,101
2,E 47 St & Park Ave,93
3,W 43 St & 10 Ave,91
4,Broadway & W 41 St,81


## Task 4 — Putting it all together
• Find the station that had the longest average trip duration during
weekends and within 500m of TimeSquare!
• Extra: create lines for trips started from stations within 500m of Times
Squares and lasted less than 2 hours. The number of trips per each
pair of stations are output as attributes of these lines

In [45]:
## Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare!
sql = '''
    SELECT  
        start_station_name
        , AVG(tripduration) AS avg_trip_duration
    FROM citibike
    WHERE 1=1
    AND SQRT(
                (111.12 * 1000 * (start_station_latitude - 40.7577))^2 + 
                (111.12 * 1000 * (end_station_longitude + 73.9857) * cos(40.7577 / 92.215))^2
            ) < 500 -- within 500m of Time Square
    AND EXTRACT(ISODOW FROM date(starttime)) IN (6, 7) -- 6: Saturday, 7: Sunday
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1
'''

In [46]:
__RAW_DF__ = pd.read_csv(StringIO.StringIO(queryCartoDB(sql)), sep=',')
__RAW_DF__.head(1)

,start_station_name,avg_trip_duration
0,12 Ave & W 40 St,1445.5


In [50]:
## Extra: create lines for trips started from stations within 500m of Times Squares and lasted less than 2 hours. 
## The number of trips per each pair of stations are output as attributes of these lines
sql = '''
    SELECT  
        start_station_name as start_node_nm
        , end_station_name as end_node_nm
        , COUNT(1) AS attr_trip_count
    FROM citibike
    WHERE 1=1
    AND SQRT(
                (111.12 * 1000 * (start_station_latitude - 40.7577))^2 + 
                (111.12 * 1000 * (end_station_longitude + 73.9857) * cos(40.7577 / 92.215))^2
            ) < 500 -- within 500m of Time Square
    AND tripduration < 2 * 60 * 60
GROUP BY 1, 2
ORDER BY 1, 2 ASC
'''

In [51]:
__RAW_DF__ = pd.read_csv(StringIO.StringIO(queryCartoDB(sql)), sep=',')
__RAW_DF__.head(7)

,start_node_nm,end_node_nm,attr_trip_count
0,11 Ave & W 41 St,5 Ave & E 29 St,5
1,11 Ave & W 41 St,6 Ave & W 33 St,5
2,11 Ave & W 41 St,8 Ave & W 52 St,2
3,11 Ave & W 41 St,Broadway & E 22 St,3
4,11 Ave & W 41 St,Broadway & W 29 St,3
5,11 Ave & W 41 St,Broadway & W 32 St,2
6,11 Ave & W 41 St,Broadway & W 36 St,1


# extra credit: make the function python 2 and 3 compatible so that it works on the  PUI2016_Python3 kernel

#### I checked that the script works for both kernels

In [49]:
### End of Notebook